# 1
## (a)  
モデルが変わったので、model関数を変更する必要がある。  
5.5節のコードの書き方であれば、それ以上に変更点はない。一方、勾配を自分で計算していて5.4節のコードなら、grad_fnの中身(dmodel_dwとdmodel_db)も新しいモデルに対応したものに変える必要がある。

## (b)
損失の計算方法や、5.5節でのbackwardの部分はコード上ではモデルが変更されたことを知り得ないようになっている。

## (c)

In [1]:
import torch
import torch.optim as optim

In [2]:
def model(t_u, w2, w1, b):
    """
    モデルだけ変わる
    """
    return w2*t_u**2 + w1*t_u + b

def loss_fn(t_p, t_c):
    """
    本の内容と同じ
    """
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    """
    本の内容と同じ
    """
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training Loss {train_loss.item():.4f}, Validation loss {val_loss.item():.4f}")
    return params

In [6]:
# Adam version
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

# train_t_un = 0.1 * train_t_u
# val_t_un = 0.1 * val_t_u

params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True) # 変数の数が一つ増えた
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)
training_loop(n_epochs = 2000, optimizer = optimizer, params = params, train_t_u = train_t_u, val_t_u = val_t_u,
              train_t_c = train_t_c, val_t_c = val_t_c)

Epoch 1, Training Loss 13651680.0000, Validation loss 2969537.5000
Epoch 2, Training Loss 11047436.0000, Validation loss 2403958.5000
Epoch 3, Training Loss 8727637.0000, Validation loss 1900049.2500
Epoch 500, Training Loss 3.2139, Validation loss 15.5588
Epoch 1000, Training Loss 2.8043, Validation loss 12.6439
Epoch 1500, Training Loss 2.6471, Validation loss 10.6635
Epoch 2000, Training Loss 2.6139, Validation loss 9.7028


tensor([ 0.0054, -0.0674, -1.2254], requires_grad=True)

訓練データの損失は下がったが、テストデータの損失は高くなった。  
過学習していることが想像される。  

## (d)  
テストデータの損失がかなり悪いことから、結果は悪化したと言える。